In [1]:
import pycalibration
pycalibration.install()

In [2]:
import numpy as np
from pycalibration import ca
import time

In [3]:
def SKCE(kernel, predictions, outcomes):

    predictions = np.array(predictions)
    outcomes = np.array(outcomes)
    n = len(outcomes)

    # Initialize SKCE
    skce_matrix = np.zeros((n,n))

    # For each pair of data points
    for i in range(n):
        for j in range(i+1,n):
            # Compute the difference between unit vectors outcomes and predictions
            diff_Yi_g_Xi = np.array([outcomes[i] - predictions[i]])
            diff_Yj_g_Xj = np.array([outcomes[j] - predictions[j]])

            # Compute the kernel matrix for the pair
            k_g_Xi_g_Xj = kernel(predictions, predictions)

            matrix_kernel = k_g_Xi_g_Xj[i,j] * np.eye(len(predictions[i]))

            skce_matrix[i,j] = ((diff_Yi_g_Xi @ matrix_kernel) @ diff_Yj_g_Xj.T)[0,0]

    skce = np.sum(skce_matrix)

    
    # Divide the SKCE by n over 2
    prefactor = (n* (n-1)) // 2
    skce /= prefactor

    return skce


def transform_outcomes(predictions, outcomes):
    #creates vector for outcomes to match predictions shape
    outcomes_minus_one = [x - 1 for x in outcomes]
    transformed_outcomes = np.zeros_like(predictions)
    transformed_outcomes[np.arange(len(predictions)), outcomes_minus_one] = 1
    return transformed_outcomes

# Use the rbf_kernel function from sklearn.metrics.pairwise
def rbf_pw(X, Y=None, length_scale=1.0):
    gamma = 1.0 / (2 * length_scale ** 2)
    return RBF_PW(X, Y, gamma=gamma)

In [4]:
from sklearn.gaussian_process.kernels import RBF as RBF_GP
from sklearn.metrics.pairwise import rbf_kernel as RBF_PW

# Instantiate the RBF kernel from sklearn.gaussian_process.kernels
rbf_gp = RBF_GP(length_scale=1)

# Use the rbf_kernel function from sklearn.metrics.pairwise
def rbf_pw(X, Y=None, length_scale=1.0):
    gamma = 1.0 / (2 * length_scale ** 2)
    return RBF_PW(X, Y, gamma=gamma)

In [20]:
rng = np.random.default_rng(1234)
import random
rng = np.random.default_rng(1234)
predictions = [rng.dirichlet((3, 2, 5)) for _ in range(40)]
outcomes = rng.integers(low=1, high=4, size=40)
predictions = np.array([[0.1,0.9],[0.9,0.1]])
outcomes = np.array([2,1])
predictions = np.array([[0.5,0.5],[0.5,0.5]])
outcomes = np.array([2,2])



#2 Datenpukte 3 Klassen
predictions = np.array([[1/3,1/3,1/3],[1/3,1/3,1/3]])
outcomes = np.array([3,1])

predictions = np.array([[0.5,0.5],[0.5,0.5],[0.5,0.5]])
outcomes = np.array([2,1,1])
#outcomes = predictions.copy()
predictions = np.array([[0.25,0.75],[0.5,0.5],[0.75,0.25]])
outcomes = np.array([2,2,1])


#print(transform_outcomes(predictions, outcomes))

# Now you can use these in your calculate_SKCE function
SKCE(rbf_pw, predictions, transform_outcomes(predictions, outcomes))
#print(SKCE(rbf_pw, predictions, outcomes))

#todo:
#2 Datenpunkte 3 Klassen
#3 Datenpunkte 2 Klassen
#Ewartungswert One Hot Encoded Vector = Wahrscheinlichkeiten


[[-0.25  0.25]] [[-0.5  0.5]]
[[0.23485327]]
[[-0.25  0.25]] [[ 0.25 -0.25]]
[[-0.0973501]]
[[-0.5  0.5]] [[ 0.25 -0.25]]
[[-0.23485327]]
[[ 0.          0.23485327 -0.0973501 ]
 [ 0.          0.         -0.23485327]
 [ 0.          0.          0.        ]]


-0.03245003262797521

In [21]:
rbf_pw(predictions, predictions)

array([[1.        , 0.93941306, 0.77880078],
       [0.93941306, 1.        , 0.93941306],
       [0.77880078, 0.93941306, 1.        ]])

In [22]:
#skce_julia = ca.SKCE(ca.RBFKernel(), unbiased=True)
skce_julia = ca.SKCE(ca.tensor(ca.RBFKernel(), ca.WhiteKernel()), unbiased = True)
skce_julia(ca.RowVecs(predictions), outcomes)

-0.03245003262797522

In [8]:
#skce_julia = ca.SKCE(ca.RBFKernel(), unbiased=True)
#skce_julia(predictions, outcomes)

In [9]:
# Measure the time taken by function1
start_time = time.time()
SKCE(rbf_gp, predictions, outcomes)
end_time = time.time()
print(f"SKCE Python took {end_time - start_time} seconds to execute.")

# Measure the time taken by function2
start_time = time.time()
skce_julia(ca.RowVecs(predictions), outcomes)
end_time = time.time()
print(f"SKCE Julia took {end_time - start_time} seconds to execute.")

[[1.75 1.25]] [[1.5 1.5]]
[[4.22735878]]
[[1.75 1.25]] [[0.25 0.75]]
[[1.07085108]]
[[1.5 1.5]] [[0.25 0.75]]
[[1.40911959]]
[[0.         4.22735878 1.07085108]
 [0.         0.         1.40911959]
 [0.         0.         0.        ]]
SKCE Python took 0.008432388305664062 seconds to execute.
SKCE Julia took 0.003968238830566406 seconds to execute.


In [10]:
import numpy as np

# Original 1D array
arr = np.array([0.05272924, -0.05272924])

# Reshape to a 2D array with a single row
arr_row = arr.reshape(1, -1)
print("Row array:")
print(arr_row)

# Reshape to a 2D array with a single column
arr_col = arr.reshape(-1, 1)
print("Column array:")
print(arr_col)


Row array:
[[ 0.05272924 -0.05272924]]
Column array:
[[ 0.05272924]
 [-0.05272924]]


In [11]:
from julia import Main
import numpy as np

def test_rbf_kernel(predictions):
    Main.eval("""
    using KernelFunctions
    rbf = SqExponentialKernel()
    """)

    kernel_matrix = np.zeros((len(predictions), len(predictions)))

    for i in range(len(predictions)):
        for j in range(i+1,len(predictions)):
            kernel_matrix[i][j] = Main.eval(f"rbf({list(predictions[i])}, {list(predictions[j])})")

    return kernel_matrix

# Test the function with two predictions
#predictions = np.array([[0.1,0.9],[0.9,0.1],[0.5,0.5]])
print(test_rbf_kernel(predictions))


[[0.         0.93941306 0.77880078]
 [0.         0.         0.93941306]
 [0.         0.         0.        ]]


In [12]:
rbf_pw(predictions, predictions)

array([[1.        , 0.93941306, 0.77880078],
       [0.93941306, 1.        , 0.93941306],
       [0.77880078, 0.93941306, 1.        ]])

In [13]:
Main.eval(f"rbf({list(predictions[0])}, {list(predictions[1])})")

0.9394130628134758

In [14]:
diff_Xi = np.array(np.array([0,1]) - np.array([0.1,0.9])).reshape(1,-1)

In [15]:
rbf_pw(predictions[0].reshape(1,-1), predictions[1].reshape(1,-1)) * np.eye(2)

array([[0.93941306, 0.        ],
       [0.        , 0.93941306]])

In [16]:
np.eye(len(predictions[0]))

array([[1., 0.],
       [0., 1.]])